In [2]:
import cv2
import numpy as np
import gradio as gr

def sky_detection(image_path):
    # Load the image
    image = cv2.imread('sky/12.jpeg') 

    # Convert to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Define the blue color range in RGB
    lower_blue = np.array([80, 80, 120]) 
    upper_blue = np.array([255, 255, 255]) 

    # Create a mask for the aky color range in the RGB image
    sky_mask = cv2.inRange(image_rgb, lower_blue, upper_blue)

    # Find the edge and remove the edge from the sky
    edges = cv2.Canny(image_rgb,40,50)
    inverted_edges = cv2.bitwise_not(edges)
    combined_mask = cv2.bitwise_and(sky_mask, inverted_edges)

    # Create an image to apply the mask to
    colored_sky = np.zeros_like(image_rgb)
    colored_sky[np.where(combined_mask == 255)] = [255, 255, 0]  # Color the sky yellow in RGB format

    # Merge the original RGB image with the colored sky
    result_rgb = cv2.addWeighted(image_rgb, 1, colored_sky, 1, 0)


    return result_rgb

iface = gr.Interface(fn=sky_detection, inputs="file", outputs="image")
iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
